# 🎬 K-Pop 안무 AI 파인튜닝 - Google Colab

## 📌 사용 방법
1. **GPU 활성화**: `Runtime > Change runtime type > GPU (T4)`
2. **순서대로 실행**: 각 셀을 위에서 아래로 실행
3. **영상 업로드**: K-pop 댄스 영상 업로드
4. **자동 파인튜닝**: 나머지는 자동으로 진행!

## ✨ 주요 기능
- ✅ K-pop 영상 자동 분석
- ✅ 포즈 추출 및 변환
- ✅ 자동 파인튜닝
- ✅ Google Drive 체크포인트 저장
- ✅ 실시간 학습 모니터링

---
## 1️⃣ 환경 설정 및 패키지 설치

In [ ]:
# GPU 확인
import torch
print("=" * 60)
print("🔍 시스템 정보")
print("=" * 60)
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA 버전: {torch.version.cuda}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️  GPU를 사용할 수 없습니다. Runtime > Change runtime type에서 GPU를 활성화하세요.")
print("=" * 60)

In [ ]:
# 필수 패키지 설치
print("📦 패키지 설치 중... (3-5분 소요)")

!pip install -q torch torchvision torchaudio
!pip install -q opencv-python-headless
!pip install -q mediapipe
!pip install -q numpy scipy scikit-learn
!pip install -q matplotlib seaborn tqdm
!pip install -q ipywidgets

print("✅ 패키지 설치 완료!")

In [ ]:
# Google Drive 마운트
from google.colab import drive
import os

drive.mount('/content/drive')

# 프로젝트 디렉토리 생성
PROJECT_DIR = '/content/drive/MyDrive/KPop_AI_Studio'
CHECKPOINT_DIR = f'{PROJECT_DIR}/checkpoints'
TRAINING_DATA_DIR = f'{PROJECT_DIR}/training_data'

os.makedirs(PROJECT_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(TRAINING_DATA_DIR, exist_ok=True)

print("✅ Google Drive 연동 완료!")
print(f"📁 프로젝트 디렉토리: {PROJECT_DIR}")

---
## 2️⃣ K-Pop 영상 업로드 및 전처리

In [ ]:
# 영상 업로드
from google.colab import files
import shutil
from pathlib import Path

print("🎬 K-Pop 댄스 영상을 업로드하세요!")
print("   권장: 10-30초 길이, 1-5개 영상")
print("   형식: MP4, MOV, AVI 등")
print("")

uploaded = files.upload()

# 업로드된 파일 저장
VIDEO_DIR = '/content/videos'
os.makedirs(VIDEO_DIR, exist_ok=True)

video_files = []
for filename in uploaded.keys():
    video_path = f"{VIDEO_DIR}/{filename}"
    with open(video_path, 'wb') as f:
        f.write(uploaded[filename])
    video_files.append(video_path)
    print(f"✅ 업로드 완료: {filename}")

print(f"\n총 {len(video_files)}개 영상 업로드 완료!")

In [ ]:
# 텍스트 캡션 입력
from IPython.display import display, HTML
import ipywidgets as widgets

print("📝 각 영상에 대한 설명을 입력하세요")
print("   예시: 'Powerful hip-hop choreography with sharp movements'")
print("   예시: 'Smooth girl group dance with elegant arm waves'")
print("")

captions = {}
caption_widgets = {}

for video_path in video_files:
    video_name = Path(video_path).name
    text_widget = widgets.Text(
        value='',
        placeholder=f'{video_name}에 대한 설명 입력...',
        description=f'{video_name[:20]}...',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='80%')
    )
    caption_widgets[video_path] = text_widget
    display(text_widget)

print("\n⬆️ 위에 각 영상의 설명을 입력한 후 다음 셀을 실행하세요!")

In [ ]:
# 캡션 저장
for video_path, widget in caption_widgets.items():
    caption = widget.value.strip()
    if not caption:
        caption = f"Dance performance from {Path(video_path).stem}"
    captions[video_path] = caption
    print(f"✅ {Path(video_path).name}: {caption}")

print(f"\n총 {len(captions)}개 캡션 저장 완료!")

In [ ]:
# 모션 추출 및 변환
import cv2
import numpy as np
import mediapipe as mp
from tqdm.notebook import tqdm
import json

print("🎯 영상에서 모션 추출 중...\n")

# MediaPipe Pose 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=2,
    smooth_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

def extract_motion_from_video(video_path, fps=20):
    """비디오에서 모션 추출"""
    cap = cv2.VideoCapture(video_path)
    video_fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    frame_interval = max(1, int(video_fps / fps))
    motion_data = []
    frame_count = 0
    
    pbar = tqdm(total=total_frames, desc=f"  {Path(video_path).name}")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_count % frame_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(frame_rgb)
            
            if results.pose_landmarks:
                landmarks = np.array([[lm.x * width, lm.y * height, lm.z * width] 
                                     for lm in results.pose_landmarks.landmark])
                motion_data.append(landmarks)
            elif motion_data:
                motion_data.append(motion_data[-1])
            else:
                motion_data.append(np.zeros((33, 3)))
        
        frame_count += 1
        pbar.update(1)
    
    cap.release()
    pbar.close()
    
    return np.array(motion_data)

def convert_to_humanml3d(motion_data):
    """MediaPipe를 HumanML3D 형식으로 변환"""
    # 간단한 매핑 (33 -> 22 joints)
    mapping = {
        0: [23, 24], 1: [11, 12], 2: [0], 3: [0],  # 중심
        4: [23], 5: [25], 6: [27], 7: [31],  # 왼쪽 다리
        8: [24], 9: [26], 10: [28], 11: [32],  # 오른쪽 다리
        12: [11], 13: [13], 14: [15], 15: [19],  # 왼쪽 팔
        16: [12], 17: [14], 18: [16], 19: [20],  # 오른쪽 팔
        20: [0], 21: [0]  # 머리
    }
    
    frames = motion_data.shape[0]
    humanml_data = np.zeros((frames, 22, 3))
    
    for frame_idx in range(frames):
        for smpl_idx, mp_indices in mapping.items():
            positions = [motion_data[frame_idx, idx] for idx in mp_indices]
            humanml_data[frame_idx, smpl_idx] = np.mean(positions, axis=0)
    
    # 정규화
    pelvis = humanml_data[:, 0:1, :]
    humanml_data = humanml_data - pelvis
    
    return humanml_data

# 모든 영상 처리
dataset = []
for idx, video_path in enumerate(video_files):
    print(f"\n[{idx+1}/{len(video_files)}] 처리 중: {Path(video_path).name}")
    
    # 모션 추출
    motion_data = extract_motion_from_video(video_path, fps=20)
    print(f"  ✓ 추출 완료: {motion_data.shape[0]}프레임")
    
    # HumanML3D 변환
    humanml_data = convert_to_humanml3d(motion_data)
    print(f"  ✓ 변환 완료: {humanml_data.shape}")
    
    # 저장
    sample_name = f"sample_{idx:04d}"
    motion_path = f"{TRAINING_DATA_DIR}/{sample_name}.npy"
    np.save(motion_path, humanml_data)
    
    dataset.append({
        'id': idx,
        'name': sample_name,
        'caption': captions[video_path],
        'video_name': Path(video_path).name,
        'frames': humanml_data.shape[0],
        'motion_path': motion_path
    })
    print(f"  ✓ 저장 완료: {motion_path}")

# 데이터셋 메타데이터 저장
dataset_path = f"{TRAINING_DATA_DIR}/dataset.json"
with open(dataset_path, 'w', encoding='utf-8') as f:
    json.dump(dataset, f, indent=2, ensure_ascii=False)

print("\n" + "="*60)
print("✅ 전처리 완료!")
print(f"📊 총 {len(dataset)}개 샘플 생성")
print(f"💾 데이터셋 저장: {dataset_path}")
print("="*60)

---
## 3️⃣ 파인튜닝 설정 및 시작

In [ ]:
# 학습 설정
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    learning_rate: float = 1e-4
    batch_size: int = 4
    num_epochs: int = 50
    save_interval: int = 500  # 500 스텝마다 저장
    log_interval: int = 50    # 50 스텝마다 로그
    max_motion_length: int = 6000  # 5분 영상 (300초 × 20fps)
    diffusion_steps: int = 1000

config = TrainingConfig()

print("⚙️ 학습 설정")
print("="*60)
print(f"Learning Rate: {config.learning_rate}")
print(f"Batch Size: {config.batch_size}")
print(f"Epochs: {config.num_epochs}")
print(f"Save Interval: {config.save_interval} steps")
print(f"Max Motion Length: {config.max_motion_length} frames (~{config.max_motion_length/20:.0f}초)")
print(f"Training Samples: {len(dataset)}")
print("="*60)

In [ ]:
# 데이터로더 생성
import torch
from torch.utils.data import Dataset, DataLoader

class MotionDataset(Dataset):
    def __init__(self, dataset_info, max_length=6000):
        self.samples = dataset_info
        self.max_length = max_length
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        motion = np.load(sample['motion_path'])
        
        # 패딩 또는 자르기
        if motion.shape[0] < self.max_length:
            padding = np.zeros((self.max_length - motion.shape[0], *motion.shape[1:]))
            motion = np.concatenate([motion, padding], axis=0)
        else:
            motion = motion[:self.max_length]
        
        return {
            'motion': torch.FloatTensor(motion),
            'caption': sample['caption'],
            'length': min(sample['frames'], self.max_length)
        }

train_dataset = MotionDataset(dataset, max_length=config.max_motion_length)
train_loader = DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

print(f"✅ 데이터로더 생성 완료")
print(f"   Batch 수: {len(train_loader)}")

In [ ]:
# 간단한 모션 생성 모델 (실제로는 MDM 사용)
# 여기서는 데모용 간단한 모델 사용
import torch.nn as nn

class SimpleMotionModel(nn.Module):
    def __init__(self, input_dim=66, hidden_dim=512, output_dim=66):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, x):
        # x: [batch, frames, joints, 3]
        batch, frames, joints, dims = x.shape
        x = x.reshape(batch, frames, -1)  # [batch, frames, joints*3]
        
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        
        decoded = decoded.reshape(batch, frames, joints, dims)
        return decoded

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleMotionModel().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.num_epochs)
criterion = nn.MSELoss()

print("✅ 모델 생성 완료")
print(f"   파라미터 수: {sum(p.numel() for p in model.parameters()):,}")
print(f"   디바이스: {device}")

In [ ]:
# 파인튜닝 시작!
from tqdm.notebook import tqdm
from datetime import datetime

print("🚀 파인튜닝 시작!")
print("="*60)

train_losses = []
global_step = 0
best_loss = float('inf')

for epoch in range(config.num_epochs):
    model.train()
    epoch_loss = 0.0
    num_batches = 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.num_epochs}")
    
    for batch in pbar:
        motion = batch['motion'].to(device)
        
        # Forward
        optimizer.zero_grad()
        output = model(motion)
        loss = criterion(output, motion)
        
        # Backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        # 로그
        epoch_loss += loss.item()
        num_batches += 1
        global_step += 1
        
        pbar.set_postfix({'loss': f"{loss.item():.4f}"})
        
        # 주기적 저장
        if global_step % config.save_interval == 0:
            checkpoint_path = f"{CHECKPOINT_DIR}/checkpoint_epoch{epoch}_step{global_step}.pt"
            torch.save({
                'epoch': epoch,
                'step': global_step,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss.item(),
            }, checkpoint_path)
            print(f"\n💾 체크포인트 저장: {checkpoint_path}")
    
    # Epoch 종료
    avg_epoch_loss = epoch_loss / num_batches
    train_losses.append(avg_epoch_loss)
    scheduler.step()
    
    print(f"\nEpoch {epoch+1} 완료 - Avg Loss: {avg_epoch_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}")
    
    # 최고 모델 저장
    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        best_model_path = f"{CHECKPOINT_DIR}/best_model.pt"
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'loss': best_loss,
        }, best_model_path)
        print(f"⭐ 최고 모델 저장! Loss: {best_loss:.4f}")

print("\n" + "="*60)
print("✅ 파인튜닝 완료!")
print(f"📊 최종 Loss: {train_losses[-1]:.4f}")
print(f"⭐ 최고 Loss: {best_loss:.4f}")
print(f"💾 체크포인트 위치: {CHECKPOINT_DIR}")
print("="*60)

---
## 4️⃣ 학습 결과 시각화

In [ ]:
# 학습 곡선
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Loss (log scale)')
plt.title('Training Loss (Log Scale)')
plt.yscale('log')
plt.grid(True)

plt.tight_layout()
plt.savefig(f'{PROJECT_DIR}/training_curve.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ 그래프 저장: {PROJECT_DIR}/training_curve.png")

---
## 5️⃣ 체크포인트 다운로드

In [ ]:
# 저장된 체크포인트 목록
import glob
from datetime import datetime

checkpoints = sorted(glob.glob(f"{CHECKPOINT_DIR}/*.pt"), 
                    key=os.path.getmtime, reverse=True)

print("📁 저장된 체크포인트:")
print("="*60)
for i, cp in enumerate(checkpoints[:10]):
    size = os.path.getsize(cp) / (1024**2)
    mtime = datetime.fromtimestamp(os.path.getmtime(cp))
    name = Path(cp).name
    print(f"{i+1:2d}. {name:40s} ({size:6.1f} MB) - {mtime.strftime('%Y-%m-%d %H:%M')}")
print("="*60)

In [ ]:
# 최고 성능 모델 다운로드
best_model_path = f"{CHECKPOINT_DIR}/best_model.pt"

if os.path.exists(best_model_path):
    print(f"💾 최고 성능 모델 다운로드 중...")
    files.download(best_model_path)
    print("✅ 다운로드 완료!")
else:
    print("❌ 최고 모델을 찾을 수 없습니다.")

---
## 📊 요약

### ✅ 완료된 작업
1. K-pop 영상 업로드 및 분석
2. 포즈 추출 및 HumanML3D 변환
3. 데이터셋 자동 생성
4. 모델 파인튜닝
5. Google Drive에 체크포인트 저장

### 📁 저장 위치
- **프로젝트**: `/content/drive/MyDrive/KPop_AI_Studio/`
- **체크포인트**: `checkpoints/`
- **학습 데이터**: `training_data/`

### 🚀 다음 단계
1. 다운로드한 `best_model.pt`를 로컬 프로젝트에서 사용
2. 더 많은 K-pop 영상으로 추가 파인튜닝
3. 프로덕션 배포

### 💡 팁
- 영상이 많을수록 더 좋은 결과
- 다양한 스타일의 안무를 포함하면 범용성 향상
- GPU 메모리 부족 시 batch_size 줄이기